# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [1]:
# Import libraries
import os
import json

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from helpers import utils
from TATSSI.download.gee_downloader import export_collection
from TATSSI.time_series.generator import Generator

In [2]:
# Area of study
with open('monterrey.geojson') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        -100.9368896484375,
        25.22233547648819
      ],
      [
        -99.755859375,
        25.22233547648819
      ],
      [
        -99.755859375,
        26.13571361317392
      ],
      [
        -100.9368896484375,
        26.13571361317392
      ],
      [
        -100.9368896484375,
        25.22233547648819
      ]
    ]
  ],
  "evenOdd": true
})


For this example we will use the MODIS MOD13A2 product. Match the band names from the [LPDAAC](https://lpdaac.usgs.gov/products/mod13a2v006/) and the [GEE](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2).

In [7]:
# Dictionary with "GEE" : "LPDAAC"
# bands = {"DayOfYear"       : "1_km_16_days_composite_day_of_the_year",
#          "EVI"             : "1_km_16_days_EVI",
#          "RelativeAzimuth" : "1_km_16_days_relative_azimuth_angle",
#          "SummaryQA"       : "1_km_16_days_pixel_reliability",
#          "sur_refl_b02"    : "1_km_16_days_NIR_reflectance",
#          "sur_refl_b07"    : "1_km_16_days_MIR_reflectance",
#          "DetailedQA"      : "1_km_16_days_VI_Quality",
#          "NDVI"            : "1_km_16_days_NDVI",
#          "SolarZenith"     : "1_km_16_days_sun_zenith_angle",
#          "sur_refl_b01"    : "1_km_16_days_red_reflectance",
#          "sur_refl_b03"    : "1_km_16_days_blue_reflectance",
#          "ViewZenith"      : "1_km_16_days_view_zenith_angle"}

In [3]:
# MOD11A2 V006
# LP DAAC https://lpdaac.usgs.gov/products/mod11a2v006/
# GEE https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A2#bands

# Dictionary with "GEE" : "LPDAAC"
bands = { "LST_Day_1km"      : "LST_Day_1km",
          "QC_Day"           : "QC_Day",
          "Day_view_time"    : "Day_view_time",
          "Day_view_angl"    : "Day_view_angl",
          "LST_Night_1km"    : "LST_Night_1km",
          "QC_Night"         : "QC_Night",
          "Night_view_time"  : "Night_view_time",
          "Night_view_angl"  : "Night_view_angl",
          "Emis_31"          : "Emis_31",
          "Emis_32"          : "Emis_32",
          "Clear_sky_days"   : "Clear_sky_days",
          "Clear_sky_nights" : "Clear_sky_nights" }

Set the download parameters

In [4]:
# Dates as YYYY-MM-DD
start_date, end_date = '2000-03-05', '2021-03-14'

# Get MODIS data
product_name, version = 'MOD11A2', '006'

# Product using GEE nomenclature
product = f'MODIS/{version}/{product_name}'

# Get GEE collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date))

# Output directory
output_dir = '/home/glopez/Projects/TATSSI/data/MOD11A2.006'

Get the data!

In [5]:
# Export image collection
datasets = export_collection(ee_collection, output_dir, bands=bands,
                             product=product, version=version,
                             scale=None, crs='EPSG:4326', region=region,
                             file_per_band=True)

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Total number of images: 966

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 1/966: 2000_03_05.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fed217bbaac35b13cb79b99a6e353e6c-4674eeb54e15e1b1a10b54a4663e3eef:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/MOD11A2.006
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 2/966: 2000_03_13.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3f376a90c97f820efb64f85b04a254fe-5e9ab66cecadd24edda0d2ae089a22b0:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/MOD11A2.006
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 3/966: 2000_03_21.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


...

INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2bd37e757ee55830e7965c92b56272f8-9f8ffdae11e6010b48a2a70cd5a3b145:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/MOD11A2.006
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 965/966: 2021_02_26.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/24608bfe6c1cacddf3f688239887058e-d00acc7370d1e646766526d3b8f3f715:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/MOD11A2.006
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 966/966: 2021_03_06.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c2dc3bcdcb16331563c5c3631f84a629-0f0f8fbd038c52716df879b4a2836fc4:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/MOD11A2.006


Create TATSSI time series.

In [6]:
# TATSSI Time Series Generator object
tsg = Generator(source_dir=output_dir, product=product_name,
                version=version, data_format='tif',
                progressBar=None, preprocessed=True)

# Create layerstacks
tsg._Generator__datasets = datasets
for dataset in tsg._Generator__datasets:
    tsg._Generator__generate_layerstack(dataset, 'tif')

# For the associated product layers, decode the corresponding QA bands
tsg._Generator__decode_qa('tif')

INFO:TATSSI.time_series.generator:Layer stack for LST_Day_1km created successfully.
INFO:TATSSI.time_series.generator:Layer stack for QC_Day created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Day_view_time created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Day_view_angl created successfully.
INFO:TATSSI.time_series.generator:Layer stack for LST_Night_1km created successfully.
INFO:TATSSI.time_series.generator:Layer stack for QC_Night created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Night_view_time created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Night_view_angl created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Emis_31 created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Emis_32 created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Clear_sky_days created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Clear_sky_nights created 

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2000_07_11.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2000_07_19.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO

INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2000_11_24.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2000_12_02.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD1

INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2001_04_15.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2001_04_23.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TA

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2001_09_14.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Day/MOD11A2.2001_09_22.006.QC_Day.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO

...

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2019_09_06.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2019_09_14.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.E

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_01_25.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_02_02.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:T

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_06_09.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_06_17.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.E

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_10_31.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Data Quality flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Emis Error flag...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer LST Error Flag...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD11A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/MOD11A2.006/QC_Night/MOD11A2.2020_11_08.006.QC_Night.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:T

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.time_series.generator:Generating MOD11A2.006 QA layer stacks for QC_Day...
INFO:TATSSI.time_series.generator:Layer stack for MODLAND created successfully.
INFO:TATSSI.time_series.generator:Layer stack for LST_Error_Flag created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Emis_Error_flag created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Data_Quality_flag created successfully.
INFO:TATSSI.time_series.generator:Generating MOD11A2.006 QA layer stacks for QC_Night...
INFO:TATSSI.time_series.generator:Layer stack for MODLAND created successfully.
INFO:TATSSI.time_series.generator:Layer stack for LST_Error_Flag created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Emis_Error_flag created successfully.
INFO:TATSSI.time_series.generator:Layer stack for Data_Quality_flag created successfully.
INFO:TATSSI.time_series.generator:Layer stack for MODLAND created successfully.
INFO:TAT